In [1]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
ids = np.load('idsMatrix22.npy')
labels = np.load('final_topics2.npy')

wordlist = np.load('wordsList.npy')
wordvec = np.load('wordVectors.npy')

ids.shape, labels.shape, wordlist.shape, wordvec.shape

((1998, 300), (1998,), (400000,), (400000, 50))

In [3]:
ids = ids[:1000,:200]

In [4]:
ids.max()

399999

In [5]:
labels = labels[:1000]

In [6]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 4
step = 1
sentences = []
next_chars = []
quote_len = ids.shape[1]

for quote in ids:
    for i in range(0, quote_len - maxlen, step):
        if quote[i] >0:
            sentences.append(quote[i: i + maxlen])
            next_chars.append(quote[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 23340


In [9]:
sentences[0:10]

[array([    7,   300,    38, 58544], dtype=int32),
 array([  300,    38, 58544,  2357], dtype=int32),
 array([   38, 58544,  2357,    10], dtype=int32),
 array([58544,  2357,    10,   645], dtype=int32),
 array([2357,   10,  645,   14], dtype=int32),
 array([ 10, 645,  14,  36], dtype=int32),
 array([ 645,   14,   36, 3130], dtype=int32),
 array([  14,   36, 3130,    4], dtype=int32),
 array([  36, 3130,    4,  682], dtype=int32),
 array([3130,    4,  682,    0], dtype=int32)]

In [11]:
chars = set()

for sentence in sentences:
    chars = set(sentence).union(set(chars))

chars = sorted(list(set(chars)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [13]:
len(char_to_int), char_to_int, int_to_char

(3812,
 {0: 0,
  3: 1,
  4: 2,
  5: 3,
  6: 4,
  7: 5,
  10: 6,
  12: 7,
  13: 8,
  14: 9,
  15: 10,
  16: 11,
  17: 12,
  18: 13,
  19: 14,
  20: 15,
  21: 16,
  22: 17,
  25: 18,
  26: 19,
  29: 20,
  30: 21,
  31: 22,
  32: 23,
  33: 24,
  34: 25,
  35: 26,
  36: 27,
  37: 28,
  38: 29,
  39: 30,
  40: 31,
  41: 32,
  42: 33,
  43: 34,
  44: 35,
  46: 36,
  47: 37,
  48: 38,
  49: 39,
  50: 40,
  51: 41,
  52: 42,
  53: 43,
  54: 44,
  55: 45,
  56: 46,
  58: 47,
  59: 48,
  60: 49,
  61: 50,
  62: 51,
  63: 52,
  64: 53,
  66: 54,
  67: 55,
  68: 56,
  69: 57,
  71: 58,
  72: 59,
  73: 60,
  74: 61,
  75: 62,
  76: 63,
  77: 64,
  78: 65,
  79: 66,
  81: 67,
  82: 68,
  83: 69,
  84: 70,
  85: 71,
  86: 72,
  87: 73,
  88: 74,
  90: 75,
  91: 76,
  92: 77,
  93: 78,
  94: 79,
  95: 80,
  96: 81,
  98: 82,
  100: 83,
  101: 84,
  102: 85,
  103: 86,
  104: 87,
  105: 88,
  106: 89,
  107: 90,
  108: 91,
  109: 92,
  110: 93,
  111: 94,
  112: 95,
  113: 96,
  114: 97,
  115: 98,
  1

In [14]:
sequences=[]
for s,sentence in enumerate(sentences):
    sequence = []
    for w, word in enumerate(sentence):
        print(s,w, word)
        sequence.append(char_to_int[word])
    sequences.append(sequence)

0 0 7
0 1 300
0 2 38
0 3 58544
1 0 300
1 1 38
1 2 58544
1 3 2357
2 0 38
2 1 58544
2 2 2357
2 3 10
3 0 58544
3 1 2357
3 2 10
3 3 645
4 0 2357
4 1 10
4 2 645
4 3 14
5 0 10
5 1 645
5 2 14
5 3 36
6 0 645
6 1 14
6 2 36
6 3 3130
7 0 14
7 1 36
7 2 3130
7 3 4
8 0 36
8 1 3130
8 2 4
8 3 682
9 0 3130
9 1 4
9 2 682
9 3 0
10 0 4
10 1 682
10 2 0
10 3 0
11 0 682
11 1 0
11 2 0
11 3 0
12 0 4
12 1 30
12 2 13288
12 3 14
13 0 30
13 1 13288
13 2 14
13 3 7
14 0 13288
14 1 14
14 2 7
14 3 636
15 0 14
15 1 7
15 2 636
15 3 586
16 0 7
16 1 636
16 2 586
16 3 4
17 0 636
17 1 586
17 2 4
17 3 336
18 0 586
18 1 4
18 2 336
18 3 5
19 0 4
19 1 336
19 2 5
19 3 4
20 0 336
20 1 5
20 2 4
20 3 30
21 0 5
21 1 4
21 2 30
21 3 30218
22 0 4
22 1 30
22 2 30218
22 3 14
23 0 30
23 1 30218
23 2 14
23 3 7
24 0 30218
24 1 14
24 2 7
24 3 179
25 0 14
25 1 7
25 2 179
25 3 3
26 0 7
26 1 179
26 2 3
26 3 214
27 0 179
27 1 3
27 2 214
27 3 16434
28 0 3
28 1 214
28 2 16434
28 3 69
29 0 214
29 1 16434
29 2 69
29 3 32
30 0 16434
30 1 69
30 2 32
3

353 3 2711
354 0 6
354 1 201534
354 2 2711
354 3 3
355 0 201534
355 1 2711
355 2 3
355 3 201534
356 0 2711
356 1 3
356 2 201534
356 3 756
357 0 3
357 1 201534
357 2 756
357 3 0
358 0 201534
358 1 756
358 2 0
358 3 0
359 0 756
359 1 0
359 2 0
359 3 0
360 0 201534
360 1 122
360 2 42
360 3 53
361 0 122
361 1 42
361 2 53
361 3 1655
362 0 42
362 1 53
362 2 1655
362 3 19
363 0 53
363 1 1655
363 2 19
363 3 162
364 0 1655
364 1 19
364 2 162
364 3 76
365 0 19
365 1 162
365 2 76
365 3 14
366 0 162
366 1 76
366 2 14
366 3 34
367 0 76
367 1 14
367 2 34
367 3 201534
368 0 14
368 1 34
368 2 201534
368 3 4158
369 0 34
369 1 201534
369 2 4158
369 3 3
370 0 201534
370 1 4158
370 2 3
370 3 22981
371 0 4158
371 1 3
371 2 22981
371 3 0
372 0 3
372 1 22981
372 2 0
372 3 0
373 0 22981
373 1 0
373 2 0
373 3 0
374 0 2357
374 1 2404
374 2 4
374 3 837
375 0 2404
375 1 4
375 2 837
375 3 127195
376 0 4
376 1 837
376 2 127195
376 3 3709
377 0 837
377 1 127195
377 2 3709
377 3 0
378 0 127195
378 1 3709
378 2 0
378 

588 0 33
588 1 201534
588 2 2174
588 3 5
589 0 201534
589 1 2174
589 2 5
589 3 201534
590 0 2174
590 1 5
590 2 201534
590 3 835
591 0 5
591 1 201534
591 2 835
591 3 4
592 0 201534
592 1 835
592 2 4
592 3 3162
593 0 835
593 1 4
593 2 3162
593 3 17
594 0 4
594 1 3162
594 2 17
594 3 7
595 0 3162
595 1 17
595 2 7
595 3 5249
596 0 17
596 1 7
596 2 5249
596 3 3222
597 0 7
597 1 5249
597 2 3222
597 3 6
598 0 5249
598 1 3222
598 2 6
598 3 201534
599 0 3222
599 1 6
599 2 201534
599 3 5236
600 0 6
600 1 201534
600 2 5236
600 3 12
601 0 201534
601 1 5236
601 2 12
601 3 1432
602 0 5236
602 1 12
602 2 1432
602 3 1515
603 0 12
603 1 1432
603 2 1515
603 3 1374
604 0 1432
604 1 1515
604 2 1374
604 3 43
605 0 1515
605 1 1374
605 2 43
605 3 346
606 0 1374
606 1 43
606 2 346
606 3 12
607 0 43
607 1 346
607 2 12
607 3 37
608 0 346
608 1 12
608 2 37
608 3 1600
609 0 12
609 1 37
609 2 1600
609 3 14
610 0 37
610 1 1600
610 2 14
610 3 2242
611 0 1600
611 1 14
611 2 2242
611 3 14107
612 0 14
612 1 2242
612 2 1

843 0 201534
843 1 992
843 2 15159
843 3 4513
844 0 992
844 1 15159
844 2 4513
844 3 5
845 0 15159
845 1 4513
845 2 5
845 3 22
846 0 4513
846 1 5
846 2 22
846 3 201534
847 0 5
847 1 22
847 2 201534
847 3 15754
848 0 22
848 1 201534
848 2 15754
848 3 3
849 0 201534
849 1 15754
849 2 3
849 3 3947
850 0 15754
850 1 3
850 2 3947
850 3 0
851 0 3
851 1 3947
851 2 0
851 3 0
852 0 3947
852 1 0
852 2 0
852 3 0
853 0 301
853 1 1655
853 2 336
853 3 19
854 0 1655
854 1 336
854 2 19
854 3 271
855 0 336
855 1 19
855 2 271
855 3 1655
856 0 19
856 1 271
856 2 1655
856 3 4
857 0 271
857 1 1655
857 2 4
857 3 242
858 0 1655
858 1 4
858 2 242
858 3 6
859 0 4
859 1 242
859 2 6
859 3 201534
860 0 242
860 1 6
860 2 201534
860 3 2237
861 0 6
861 1 201534
861 2 2237
861 3 5
862 0 201534
862 1 2237
862 2 5
862 3 19
863 0 2237
863 1 5
863 2 19
863 3 12
864 0 5
864 1 19
864 2 12
864 3 1132
865 0 19
865 1 12
865 2 1132
865 3 1655
866 0 12
866 1 1132
866 2 1655
866 3 6
867 0 1132
867 1 1655
867 2 6
867 3 271
868 0 

1086 2 936
1086 3 0
1087 0 346
1087 1 936
1087 2 0
1087 3 0
1088 0 936
1088 1 0
1088 2 0
1088 3 0
1089 0 3305
1089 1 271
1089 2 43
1089 3 36
1090 0 271
1090 1 43
1090 2 36
1090 3 1655
1091 0 43
1091 1 36
1091 2 1655
1091 3 214
1092 0 36
1092 1 1655
1092 2 214
1092 3 83
1093 0 1655
1093 1 214
1093 2 83
1093 3 44
1094 0 214
1094 1 83
1094 2 44
1094 3 10137
1095 0 83
1095 1 44
1095 2 10137
1095 3 33
1096 0 44
1096 1 10137
1096 2 33
1096 3 5496
1097 0 10137
1097 1 33
1097 2 5496
1097 3 575
1098 0 33
1098 1 5496
1098 2 575
1098 3 36
1099 0 5496
1099 1 575
1099 2 36
1099 3 4
1100 0 575
1100 1 36
1100 2 4
1100 3 1655
1101 0 36
1101 1 4
1101 2 1655
1101 3 336
1102 0 4
1102 1 1655
1102 2 336
1102 3 0
1103 0 1655
1103 1 336
1103 2 0
1103 3 0
1104 0 336
1104 1 0
1104 2 0
1104 3 0
1105 0 936
1105 1 12
1105 2 14
1105 3 588
1106 0 12
1106 1 14
1106 2 588
1106 3 219
1107 0 14
1107 1 588
1107 2 219
1107 3 5
1108 0 588
1108 1 219
1108 2 5
1108 3 79132
1109 0 219
1109 1 5
1109 2 79132
1109 3 5376
1110 0

...

22402 3 56
22403 0 14
22403 1 1663
22403 2 56
22403 3 10247
22404 0 1663
22404 1 56
22404 2 10247
22404 3 10
22405 0 56
22405 1 10247
22405 2 10
22405 3 641
22406 0 10247
22406 1 10
22406 2 641
22406 3 4018
22407 0 10
22407 1 641
22407 2 4018
22407 3 827
22408 0 641
22408 1 4018
22408 2 827
22408 3 5
22409 0 4018
22409 1 827
22409 2 5
22409 3 201534
22410 0 827
22410 1 5
22410 2 201534
22410 3 76
22411 0 5
22411 1 201534
22411 2 76
22411 3 873
22412 0 201534
22412 1 76
22412 2 873
22412 3 53
22413 0 76
22413 1 873
22413 2 53
22413 3 408
22414 0 873
22414 1 53
22414 2 408
22414 3 4
22415 0 53
22415 1 408
22415 2 4
22415 3 88
22416 0 408
22416 1 4
22416 2 88
22416 3 14
22417 0 4
22417 1 88
22417 2 14
22417 3 4
22418 0 88
22418 1 14
22418 2 4
22418 3 2025
22419 0 14
22419 1 4
22419 2 2025
22419 3 308
22420 0 4
22420 1 2025
22420 2 308
22420 3 13
22421 0 2025
22421 1 308
22421 2 13
22421 3 336
22422 0 308
22422 1 13
22422 2 336
22422 3 2695
22423 0 13
22423 1 336
22423 2 2695
22423 3 5
224

22610 2 208
22610 3 5
22611 0 14
22611 1 208
22611 2 5
22611 3 8532
22612 0 208
22612 1 5
22612 2 8532
22612 3 43
22613 0 5
22613 1 8532
22613 2 43
22613 3 36
22614 0 8532
22614 1 43
22614 2 36
22614 3 2968
22615 0 43
22615 1 36
22615 2 2968
22615 3 10
22616 0 36
22616 1 2968
22616 2 10
22616 3 201534
22617 0 2968
22617 1 10
22617 2 201534
22617 3 16434
22618 0 10
22618 1 201534
22618 2 16434
22618 3 1077
22619 0 201534
22619 1 16434
22619 2 1077
22619 3 18
22620 0 16434
22620 1 1077
22620 2 18
22620 3 1809
22621 0 1077
22621 1 18
22621 2 1809
22621 3 98
22622 0 18
22622 1 1809
22622 2 98
22622 3 26
22623 0 1809
22623 1 98
22623 2 26
22623 3 25329
22624 0 98
22624 1 26
22624 2 25329
22624 3 125
22625 0 26
22625 1 25329
22625 2 125
22625 3 7
22626 0 25329
22626 1 125
22626 2 7
22626 3 300
22627 0 125
22627 1 7
22627 2 300
22627 3 14
22628 0 7
22628 1 300
22628 2 14
22628 3 8600
22629 0 300
22629 1 14
22629 2 8600
22629 3 3
22630 0 14
22630 1 8600
22630 2 3
22630 3 336
22631 0 8600
22631

22856 3 0
22857 0 114
22857 1 0
22857 2 0
22857 3 0
22858 0 96
22858 1 3
22858 2 201534
22858 3 1422
22859 0 3
22859 1 201534
22859 2 1422
22859 3 74
22860 0 201534
22860 1 1422
22860 2 74
22860 3 201534
22861 0 1422
22861 1 74
22861 2 201534
22861 3 7220
22862 0 74
22862 1 201534
22862 2 7220
22862 3 3
22863 0 201534
22863 1 7220
22863 2 3
22863 3 336
22864 0 7220
22864 1 3
22864 2 336
22864 3 5
22865 0 3
22865 1 336
22865 2 5
22865 3 214
22866 0 336
22866 1 5
22866 2 214
22866 3 14
22867 0 5
22867 1 214
22867 2 14
22867 3 59
22868 0 214
22868 1 14
22868 2 59
22868 3 546
22869 0 14
22869 1 59
22869 2 546
22869 3 201534
22870 0 59
22870 1 546
22870 2 201534
22870 3 1422
22871 0 546
22871 1 201534
22871 2 1422
22871 3 6545
22872 0 201534
22872 1 1422
22872 2 6545
22872 3 13
22873 0 1422
22873 1 6545
22873 2 13
22873 3 201534
22874 0 6545
22874 1 13
22874 2 201534
22874 3 732
22875 0 13
22875 1 201534
22875 2 732
22875 3 779
22876 0 201534
22876 1 732
22876 2 779
22876 3 4
22877 0 732
22

23065 2 14
23065 3 134
23066 0 1877
23066 1 14
23066 2 134
23066 3 2910
23067 0 14
23067 1 134
23067 2 2910
23067 3 6
23068 0 134
23068 1 2910
23068 2 6
23068 3 201534
23069 0 2910
23069 1 6
23069 2 201534
23069 3 104
23070 0 6
23070 1 201534
23070 2 104
23070 3 112
23071 0 201534
23071 1 104
23071 2 112
23071 3 19
23072 0 104
23072 1 112
23072 2 19
23072 3 7
23073 0 112
23073 1 19
23073 2 7
23073 3 5553
23074 0 19
23074 1 7
23074 2 5553
23074 3 7742
23075 0 7
23075 1 5553
23075 2 7742
23075 3 0
23076 0 5553
23076 1 7742
23076 2 0
23076 3 0
23077 0 7742
23077 1 0
23077 2 0
23077 3 0
23078 0 41
23078 1 15
23078 2 6
23078 3 55
23079 0 15
23079 1 6
23079 2 55
23079 3 3855
23080 0 6
23080 1 55
23080 2 3855
23080 3 697
23081 0 55
23081 1 3855
23081 2 697
23081 3 3257
23082 0 3855
23082 1 697
23082 2 3257
23082 3 11678
23083 0 697
23083 1 3257
23083 2 11678
23083 3 2153
23084 0 3257
23084 1 11678
23084 2 2153
23084 3 32703
23085 0 11678
23085 1 2153
23085 2 32703
23085 3 39
23086 0 2153
2308

23276 2 336
23276 3 79289
23277 0 49
23277 1 336
23277 2 79289
23277 3 405
23278 0 336
23278 1 79289
23278 2 405
23278 3 4
23279 0 79289
23279 1 405
23279 2 4
23279 3 3899
23280 0 405
23280 1 4
23280 2 3899
23280 3 2412
23281 0 4
23281 1 3899
23281 2 2412
23281 3 59
23282 0 3899
23282 1 2412
23282 2 59
23282 3 20
23283 0 2412
23283 1 59
23283 2 20
23283 3 113
23284 0 59
23284 1 20
23284 2 113
23284 3 47
23285 0 20
23285 1 113
23285 2 47
23285 3 191
23286 0 113
23286 1 47
23286 2 191
23286 3 1479
23287 0 47
23287 1 191
23287 2 1479
23287 3 191
23288 0 191
23288 1 1479
23288 2 191
23288 3 1479
23289 0 1479
23289 1 191
23289 2 1479
23289 3 1071
23290 0 191
23290 1 1479
23290 2 1071
23290 3 81
23291 0 1479
23291 1 1071
23291 2 81
23291 3 465
23292 0 1071
23292 1 81
23292 2 465
23292 3 2412
23293 0 81
23293 1 465
23293 2 2412
23293 3 59
23294 0 465
23294 1 2412
23294 2 59
23294 3 20
23295 0 2412
23295 1 59
23295 2 20
23295 3 5
23296 0 59
23296 1 20
23296 2 5
23296 3 81
23297 0 20
23297 1 5


In [15]:
sequences = np.asarray(sequences)

In [16]:
sequences.shape, sequences.max()

((23340, 4), 3811)

In [17]:
np.save('sequences.npy',sequences)

In [18]:
next_seq = []
for w, word in enumerate(next_chars):
    next_seq.append(char_to_int[word])

In [19]:
next_seq = np.asarray(next_seq)

In [20]:
np.save('next_seq.npy',next_seq)

In [21]:
next_seq.shape, next_seq.max()

((23340,), 3811)

In [70]:
max_word = np.asarray(next_seq.max())

In [22]:
from keras.utils import to_categorical

In [26]:
# reshape X to be [samples, time steps, features]
X = np.reshape(sequences, (len(sequences), maxlen, 1))
# normalize
X = X / float(next_seq.max())
# one hot encode the output variable
y = to_categorical(next_seq)

In [32]:
y.shape, X.shape

((23340, 3812), (23340, 4, 1))

In [33]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))



model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [34]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [35]:
text = np.ndarray.flatten(sequences)

In [36]:
text.shape

(93360,)

In [117]:
x_pred = 0
generated = 0
sentence = 0
def on_epoch_end(epoch, logs):
    global x_pred
    global sentence 
    global generated
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        print(sentence)
        generated.join([str(wordlist[[int_to_char[value]]]).join(' ') for value in sentence])
        print('----- Generating with seed: %s'%sentence)
#         sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.reshape(sentence,(1, maxlen, 1))
            x_pred = x_pred / max_word

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = wordlist[int_to_char[next_index]]

            generated.join(str(next_char))
            sentence = np.append(sentence[1:],next_index)

            sys.stdout.write(next_char)
            sys.stdout.write(" ")
            sys.stdout.flush()
        print()

In [118]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(X, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
23340/23340 [==============================] - 18s 784us/step - loss: 4.9781

----- Generating text after Epoch: 0
----- diversity: 0.2
[   1 2628    3    1]
----- Generating with seed: [   1 2628    3    1]
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

do you is death game is 0 live time has but who had hurts and 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 the 0 0 0 0 0 0 0 0 0 the that friends it 0 0 0 0 0 0 0 0 all 0 think of 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 should you life have 0 stage and music 0 dead own this the the of life our but 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 unk 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 of 0 0 it the it like the to a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 me of every that the death we a father by 0 0 0 0 0 a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 being 0 the being of not going were be highly is them 0 wanted of 0 0 0 0 0 0 0 0 0 0 0 0 0 0 it 0 0 be 0 death been be a 0 sick young that a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
----- divers

of death row 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
----- diversity: 0.5
[ 969  272 3548  969]
----- Generating with seed: [ 969  272 3548  969]
to death trying life trouble of 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

walkers of hell the on death all you a live the years or regarding so that continue by that 0 0 0 0 0 these you think the death to his 0 0 0 0 0 0 0 not 0 0 0 0 0 0 0 0 that 0 equilibrium these 0 0 0 0 0 0 0 0 0 0 0 a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 by 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 on 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 human youre every be a her 0 going of a 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 lets 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 its 0 human out if find about the 0 to 0 0 0 0 0 0 0 filled it the i to a frightened on his as 0 there phd can not death not it the death 0 the light of my science or is 0 whom 0 0 0 0 0 0 0 0 0 0 0 part 0 are live to 0 0 that 0 away human on a no the law have my than more is while 0 the to a 0 0 being that be that 0 0 have 0 0 going with i 0 always in 0 0 0 all 0 a is the right of my political that death and 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 what 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 it 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: 